在推荐系统届最最有名的算法就属协同过滤了，协同过滤包括：基于记忆的协同过滤（Memory-Based）、基于模型的协同过滤（Model-based）。
<img src="./imgs/img03.png" width = "400" height = "400" />




基于记忆的协同过滤算法的原理是：记住每个人消费过的东西，给他推荐相似的东西或相似的人消费的东西。

基于模型的协同过滤算法的原理是：从用户关系矩阵中学习到一个模型，从而将矩阵空白处填满。

### 基于用户的协同过滤

1.算法过程

    1.1 准备用户物品关系矩阵
        1.1.1 向量维度是物品个数
        1.1.2 向量是稀疏的
        1.1.3 向量维度的取值可以是简单的0或1，1喜欢，0没有行为
    1.2 计算两两用户之间的相似度
    1.3 为每一个用户产生推荐结果
2.可能遇到的问题
    
    2.1 只有原始行为日志，需要从中构造出矩阵，怎么做？
        2.1.1 采用系数矩阵存储格式CSR、COO（numpy、spark都支持）
    2.2 若用户向量很长，计算相似度耗时很久，怎么办？
        2.2.1 对向量采样计算，两个100维计算出相似度是0.7，现在考虑精度的损失，不从一百维计算，随机从中取出10维计算，得到相似度为0.72，在不损失太多精度的情况下，计算复杂度降低10倍这还是很值得的。这个算法由Twitter提出，叫做DIMSUM算法，spark中支持
        2.2.2 向量化计算，numpy支持向量化运算
    2.3 若用户量很大，两两计算用户相似度也是一个大坑，怎么办？
        2.3.1 将相似度计算拆分成Map Reduce任务，将原始矩阵Map成键为用户对，值为用户对同一物品的评分之积，Reduce阶段对这些乘积再求和，Map Reduce任务结束后再对这些值归一化
        2.3.2 不用基于用户的协同过滤
    2.4 在计算推荐时，看上去要为用户计算他和每一个物品的分数，怎么办？
        2.4.1 只有相似用户喜欢过的物品需要计算
        2.4.2 把计算过程拆成Map Reduce任务
3.改进
    
    3.1 惩罚对热门物品的喜欢程度；热门物品很难反应用户真实兴趣，也可能是被煽动或随便点击
    3.2 增加喜欢程度的时间衰减；一般是一个指数函数，指数是负数，值和喜欢行为发生时间正相关即可
4.应用场景
    
    4.1 推荐物品；首页的“猜你喜欢”
    4.2 推荐用户；例如某社交平台看到”相似粉丝“，”和你口味类似的人“等等

### 基于物品的协同过滤

#### 1. 为什么需要基于用户的协同过滤

1.基于用户的协同过滤有哪些问题？
    
    1.1 用户数量比较大，计算吃力，成为瓶颈
    1.2 用户口味变化快，兴趣迁移问题难以反映过来
    1.3 数据稀疏，用户和用户之间的共同消费行为比较少，一般都是热门物品，都发现用户兴趣帮助不大
2.基于物品的算法怎么样？
    
    2.1 若物品数量少于用户，计算物品之间的相似度不会成为瓶颈
    2.2 物品之间的相似度比较静态，它们的变化速度没用户口味变化快，解决了用户迁移问题
    2.3 物品对应的消费者数量大，计算物品之间的相似度的稀疏性要好过用户之间的相似性的
    
#### 2. 如何做基于物品的协同过滤？

1.算法过程
    
    1.1 构建用户物品的关系矩阵，矩阵元素可以是用户的消费行为，也可以是消费后的评价，还可以是消费行为的某种量化如次数、时间、费用等。
    1.2 根据用户物品关系矩阵计算物品之间的相似性，得到物品的相似度矩阵
        1.2.1 一般使用余弦相似度，分母为两个物品向量的长度，分子为两个向量的点积，即相同位置的元素值相乘再求和
<img src="./imgs/img04.png" width = "200" height = "200" />

    1.3 预测用户u对物品i的分数
        1.3.1 遍历用户u评分过的所有物品，例如一共有m个，每个物品和物品i的相似度乘以用户的评分，这样加权求和后，除去所有相似度的综合，得到一个加权平均评分，作为用户u对物品i的分数
<img src="./imgs/img05.png" width = "200" height = "200" />

2.改进
    
    2.1 物品中心化
        2.1.1 把用户物品关系矩阵中的分数，减去物品分数的均值，目的是去掉粉丝群体的非理性因素
    2.2 用户中心化
        2.2.1 把用户物品关系矩阵中的分数，减去用户分数的均值，目的是避免不同用户评分标准不同的情况，去掉主观成分
        
3.应用场景

    3.1 TopK推荐
        3.1.1 形式上属于”猜你喜欢”这样的，当用户访问首页时，汇总“用户已经消费国的物品的相似物品”，按照汇总后的分数从高到低推荐
    3.2 相关推荐
        3.2.1 当用户访问物品详情页或消费完之后的结果页，获取物品的相似物品进行推荐，这就是“看了又看”，“买了又买”

### 相似度公式

- 1.欧式距离
    - 1.1 每个坐标上的取值相减，求平方和，最后输出求平方根
    <img src="./imgs/img06.png" width = "200" height = "200" />
    - 1.2 若希望结果在[0, 1]之间，则给出常用的转化公式
    <img src="./imgs/img07.png" width = "80" height = "80" />
- 2.余弦相似度
    - 2.1 分子积分，分母两向量平方和后开方相乘
    <img src="./imgs/img08.png" width = "180" height = "180" />
- 3.皮尔逊相似度
    - 3.1 先对向量做了中心化，向量p和q各自减去向量的均值后，再计算余弦相似度，皮尔逊相关度的计算结果范围是-1到1，-1表示负相关，1表示正相关，实际度量的两个变量是否同增共减。皮尔逊相关度不适合计算布尔向量之间的相关度
    <img src="./imgs/img09.png" width = "250" height = "250" />
- 4.Jaccard相似度
    - 4.1 两个向量交集元素在并集中的比例，专门用于布尔值(0, 1)向量
    <img src="./imgs/img10.png" width = "120" height = "120" />


### 基于模型的协同过滤

#### 1.为什么需要基于模型的协同过滤？
- 1.近邻模型的问题
    - 1.1 物品之间存在相关性，信息量不随着向量维度的增加而增加
    - 1.2 矩阵元素稀疏，计算结果不稳定，增减一个向量维度，会导致近邻结果差异很大
    - 1.3 这两个问题都可以通过矩阵分解模型解决
        
#### 2.什么是基于模型的协同过滤？
- 1.直观地说，就是把原来的大矩阵，近似分解成两个小矩阵的点积，一个矩阵是代表用户偏好的用户隐因子向量组成，另一个矩阵是代表物品语义主题的隐因子向量组成。在实际推荐时不再使用大矩阵，而是使用分解得到的两个小矩阵，这两个小矩阵相乘后得到的矩阵，维度和用户评分一致。
- 2.物理意义，矩阵分解就是把用户和物品映射到k维空间中，这个k维空间不是我们直接看到的，也不一定具有非常好的解释性，每一个维度也没有名字，所以叫做隐因子。矩阵分解模型又叫做隐因子模型。
   
#### 3.如何做基于模型的协同过滤？

- 1.SVD
    - 1.1 学习过程
        - 1.1.1 准备好用户评分矩阵，每一条数据样本看做一个训练样本
        - 1.1.2 对分解后的U和V矩阵随机初始化值
        - 1.1.3 用U和V矩阵点积计算预测后的分数
        - 1.1.4 计算预测分数和实际分数误差
        - 1.1.5 按照梯度下降的方向更新U和V中的值
        - 1.1.6 重复1.1.3和1.1.5，直到达到终止条件
    - 1.2 损失函数
    <img src="./imgs/img11.png" width = "300" height = "300" />
    - 1.3 优化算法
        - 1.3.1 增加偏置信息；如图，从左到右依次是全局平均分、物品评分偏置、用户评分偏置、用户和物品之间的兴趣偏好；此时目标函数为
        <img src="./imgs/img12.png" width = "200" height = "200" />
        <img src="./imgs/img13.png" width = "400" height = "400" />

- 2.SVD++
    - 2.1 为什么需要SVD++？
        - 2.1.1 有些用户评分比较少，能不能用隐反馈弥补这块的不足呢？另外，用户的个人属性，比如性别、年龄，也能否加入模型弥补冷启动的不足呢？
    - 2.2 什么是SVD++？
        - 2.2.1 相比于SVD模型，SVD++结合了用户的隐反馈行为和用户属性
    - 2.3 如何学习SVD++模型？
        - 2.3.1 相比于SVD，在用户向量部分增加了隐反馈向量和用户属性向量
        <img src="./imgs/img14.png" width = "400" height = "400" />

- 3.考虑时间因素，有如下几种做法：
    - 3.1 对评分按照时间加权，让久远的评分更趋于平均值
    - 3.2 对评分时间划分区间，不同的时间区间内分别学习出隐因子向量，使用时按照区间使用对应的隐因子来计算
    - 3.3 对特殊的期间，如节日、周末等训练对应的隐因子向量

- 4.ALS（交替最小二乘求矩阵分解）
    - 4.1 如何学习交替最小二乘？
        - 4.1.1 有了目标函数后，我们可以使用优化算法求参数，常用的优化有SGD、ALS（交替最小二乘）
        <img src="./imgs/img15.png" width = "200" height = "200" />
        ALS的任务是找到两个矩阵P和Q，让他们相乘后约等于原矩阵R，
        假定我们知道Q，那P可以通过R乘以Q的逆矩阵求解
        <img src="./imgs/img16.png" width = "200" height = "200" />
            - 4.1.1.1 初始化随机矩阵Q里面的值
            - 4.1.1.2 把矩阵Q当做已知的，直接用线性代数求解矩阵P
            - 4.1.1.3 得到矩阵P后，把P当做已知，故技重施，回去求解矩阵Q
            - 4.1.1.4 上面两个过程交替进行，一直到误差可以接受为止
    - 4.2 ALS的优点？
        - 4.2.1 交替进行的其中一步，假设其中一个求解另一个，有优化的参数是很容易并行的
        - 4.2.2 在不那么稀疏的数据集上，交替最小二乘比随机梯度下降更快得到结果
        
- 5.隐式反馈怎么处理？
    - 5.1 为什么要处理隐式反馈？
        - 5.1.1 相比于“预测用户会打多少分”，“预测用户会不会浏览”更有意义，而且浏览数据远多于打分数据
        - 5.1.2 如果预测用户行为看成一个二分类，猜用户会不会做某件事，而实际收集来的数据只有一类；若用户干了某件事，而用户明确不干某件事在数据上却没有明确表达。这就是one-class问题，one-class是隐反馈问题的通常特点
    - 5.2 如何做隐式反馈的矩阵分解？
        - 5.2.1 采用交替最小二乘的改进，加权交替最小二乘
        - 5.2.2 算法过程
            - 5.2.2.1 若用户对物品隐反馈则认为评分是0
            - 5.2.2.2 若用户对物品至少一次隐反馈则认为评分是1，次数作为该评分的置信度
        - 5.2.3 目标函数
            <img src="./imgs/img17.png" width = "300" height = "300" />
            Cui就是置信度，计算误差时考虑反馈次数，次数越多，就越可信；
            一般置信度这样计算，Cui=1+αC，α是超参数，需要调参，默认40，能达不到差不多效果，C是次数
    - 5.3 问题
        - 5.3.1 那些没有反馈的缺失值很多，需要采用负样本采样挑一部分作为负类别样本
            - 5.3.1.1 随机均匀采样和正类别一样多（不靠谱）
            - 5.3.1.2 按照物品的热门程度采样（实践中经过了检验，因为一个用户越热门，用户越可能知道它的存在，若用户还没对它有反馈就表明这是一个真正的负样本）
       - 5.3.2 用户和物品的隐因子向量直接计算点积复杂度会很好
           - 5.3.2.1 专门使用一些数据结构存储物品的隐因子向量，从而实现从用户向量可以返回最相似的k个物品，用户向量则可以存在内存数据中，这样用户访问时，实时产生推荐结果
           - 5.3.2.2 先对物品的隐因子向量做聚类，然后逐一计算用户和每个聚类中心的推荐分数，先给用户推荐聚类，然后从聚类中挑选少数几个物品作为最终的推荐结果。这样做还能控制每个类选择的数量，提高推荐的多样性。